In [1]:
import logging
from collections import namedtuple
import random
import itertools
import math

In [2]:
PROBLEM_SIZE = [5, 10, 20, 100, 500, 1000]
N=0
OFFSPRING_SIZE=0 # it's dynamic
TOURNAMENT_SIZE=10
POPULATION_SIZE = 100
NUM_GENERATIONS = 250

### Problem definition:

In [3]:
def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

## Function of genetic algorithm

In [4]:

Individual = namedtuple("Individual", ["genome", "fitness"])

def fitness(genome):
    ''' fitness provides a tuple made by the elements on their own and the total count of them (negativized to better obtain the maximization function) '''
    count=0
    for l in genome:
        count+=len(l)
    return (set(itertools.chain(*genome)).__len__(),-count)


def tournament(population, tournament_size=TOURNAMENT_SIZE):
    return max(random.choices(population, k=tournament_size), key=lambda i: i.fitness)


def cross_over(g1, g2):
    ''' basic cross_over function, it just substitute parts of a genome '''    
    cut = random.randint(0, len(g1))
    c1=g2[:cut]+g1[cut:]
    c2=g1[:cut] + g2[cut:]
    r1=fitness(c1)
    r2=fitness(c2)
    if r1>r2: return c1
    else: return c2


def cross_over_twopoints(g1, g2):
    ''' cross_over functions with two cut points, be careful to the major computational cost'''
    cut1 = random.randint(0, len(g1))
    cut2 = random.randint(0, len(g1))

    try1=remove_identical(g1[:min(cut1,cut2)] + g2[min(cut1,cut2):max(cut1,cut2)] + g1[max(cut1,cut2):])
    try2=remove_identical(g2[:min(cut1,cut2)] + g1[min(cut1,cut2):max(cut1,cut2)] + g2[max(cut1,cut2):])
    r1=fitness(try1)
    r2=fitness(try2)
    if r1>r2: return try1
    else: return try2

def mutation(g,generator):
    ''' this functions provides an equal probability of removing or adding a new gene'''
    #remove element
    if random.random()>0.5 and len(g)>1:
        g.pop(random.randint(0,len(g)-1))
    #Add element
    else:
        for i in generator:
            m=random.choice(generator)
            if g.__contains__(m):
                continue
            else:
                g.append(m)
                break
    return g 

def solvable(Gen):
    ''' solvable checks if the given problem is solvable or not '''
    if set(itertools.chain(*Gen))!=set(range(0,N)):
        return False
    else:
        return True

def remove_identical(Lista):
    ''' remove_identical avoids unuseful computations by removing identicals from a list '''
    L = []
    for l in Lista:
        if L.__contains__(l):
            continue
        else:
            L.append(l)
    return L

# Genetic Algorithm

## Evolution

In [5]:
def Evolution(population, generator,fitness_log):
    for g in range(NUM_GENERATIONS):
        offspring = list()
        for i in range(OFFSPRING_SIZE):
            if random.random() < 0.2:
                p = tournament(population)
                o = mutation(p.genome,generator)
            else:
                p1 = tournament(population)
                p2 = tournament(population)
                o = cross_over(p1.genome, p2.genome)
            f = fitness(o)
            fitness_log.append((g + 1, f))
            offspring.append(Individual(o, f))
        population += offspring
        population = sorted(population, key=lambda i: i.fitness, reverse=True)[:N]

    
    logging.info(f" For N={N} -> W={-population[0][1][1]} BLOAT={int((-population[0][1][1]-N)*100/N)} %")


## Initial Population

In [6]:
logging.getLogger().setLevel(logging.INFO)

population = list()
for n in PROBLEM_SIZE:

    N=n
    
    OFFSPRING_SIZE = round(N*10)
    TOURNAMENT_SIZE = math.ceil(N/5)
    generator = problem(N,42)

    #Check if a solution exists
    if(solvable(generator)==False):
        print("problem not solvable")
        exit(-1)
    
    #remove duplicate list
    generator = remove_identical(generator)
    
    for genome in generator:
        l=list()
        l.append(genome)
        population.append(Individual(l, fitness(l)))

    fitness_log = [(0, i.fitness) for i in population]

    logging.info(f"Starting calculating for N={N} GEN MAX={NUM_GENERATIONS}")
    Evolution(population, generator, fitness_log)

INFO:root:Starting calculating for N=5 GEN MAX=250
INFO:root: For N=5 -> W=5 BLOAT=0 %
INFO:root:Starting calculating for N=10 GEN MAX=250
INFO:root: For N=10 -> W=11 BLOAT=10 %
INFO:root:Starting calculating for N=20 GEN MAX=250
INFO:root: For N=20 -> W=24 BLOAT=20 %
INFO:root:Starting calculating for N=100 GEN MAX=250
INFO:root: For N=100 -> W=192 BLOAT=92 %
INFO:root:Starting calculating for N=500 GEN MAX=250
INFO:root: For N=500 -> W=1394 BLOAT=178 %
INFO:root:Starting calculating for N=1000 GEN MAX=250
INFO:root: For N=1000 -> W=3397 BLOAT=239 %
